### Loading of OpenAI API Key from `.env` file

In [1]:
import dotenv
dotenv.load_dotenv()

True

### A simple parameterized LLM call

In [2]:
from langchain.chat_models import ChatOpenAI  # Chat API
# from langchain.llms import OpenAI  # Legacy text-only API

# Prompt Abstraction
from langchain.prompts import BaseChatPromptTemplate  # not needed, this is simply the base-type for Chat Prompt Templates
from langchain.prompts import ChatPromptTemplate

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo", # default
                   # model="gpt-4", # GPT4
)

In [4]:
# Call model directly
model.invoke("Tell me joke about cats")

AIMessage(content="Why don't cats play poker in the wild?\n\nToo many cheetahs!")

In [5]:
prompt: BaseChatPromptTemplate = ChatPromptTemplate.from_template("""Tell me a joke about {topic}""")

In [6]:
# Demonstrate instantiation of prompt templates by passing required parameters
prompt.invoke({"topic": "tigers"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a joke about tigers')])

Important insight:

All the essential building blocks provided by LangChain (models, prompts, retrievers) have a common interface.
 
They all support, amongst others, the following methods:

```python
# invoke
# stream
# batch
```

In [7]:
model.invoke(prompt.invoke({"topic": "tigers"}))

AIMessage(content='Why was the tiger late to the party?\n\nBecause he got caught up in a "spotted" traffic jam!')

### LangChain Expression Language (LCEL)

We can express the same sequence more elegantly using LCEL syntax. 

In [8]:
chain = prompt | model

In [9]:
result = chain.invoke({"topic": "goldfish"})
result

AIMessage(content="Why don't goldfish like to play basketball?\n\nBecause they're afraid of getting caught in the net!")

If we wish to obtain only the text, we can do `result.content`.  
But there is a more elegant way to include this transformation directly as part of the chain.

In [10]:
from langchain.schema import StrOutputParser

parser = StrOutputParser()

In [11]:
from langchain.schema import AIMessage

ai_message = AIMessage(content="This is a test.")
ai_message

AIMessage(content='This is a test.')

In [12]:
# Parsers support the same interface, we `invoke` them
parser.invoke(ai_message)

'This is a test.'

In [13]:
chain = prompt | model | parser

In [14]:
print(chain.invoke({"topic": "lemons"}))

Why did the lemon go to the doctor?

Because it wasn't peeling well!


### Passing in strings rather than mappings

So far we always called the chain by invoking it with a dictionary.  
What, if we simply wish to pass in a text?

In [15]:
try:
  chain.invoke("lemons")
except TypeError as err:
  print(err)
# We receive a `TypeError`. This is expected.

Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.


We receive an error as the chain expects a mapping type.  
We can verify this by looking at the expected input schema:

In [16]:
chain.input_schema.schema_json()

'{"title": "PromptInput", "type": "object", "properties": {"topic": {"title": "Topic", "type": "string"}}}'

The schema is displayed in `JSON Schema` format. 

We can read this as "The input schema for the first segment of the chain, titled _PromptInput_, expects to receive an `object` (i.e. a Python dict)^ with one property (i.e. attribute) `topic` which in turn is of type `string`. So the input should be of the form `{"topic": my_topic: str}`."

^Note:  
In JSON Schema, specifying `"type": "object"` means that the value is expected to be a JSON object. An object in JSON is a collection of key/value pairs, where each key is a string, and the value can be of any type, including another object. A JSON object is thus analogous to a dictionary in Python. 

So, in order to call out chain with a string, we need a way to transform the input string into a mapping type that the prompt knows how to consume.

We can do this as follows:

In [17]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [18]:
chain = RunnableParallel(topic=RunnablePassthrough()) | prompt | model | parser

In [19]:
chain.invoke("goldfish")

'Why did the goldfish bring a suitcase to the party?\n\nBecause he wanted to travel in "fin" style!'

### Including more than one variable

In [20]:
prompt = ChatPromptTemplate.from_template("""
Tell me something about {topic}.

Answer in {language}.
""")

In [21]:
chain = prompt | model | parser


In [22]:
# Familiar way of invoking the chain
chain.invoke({"topic": "lemons", "language": "Dutch"})

'Limoenen zijn citrusvruchten die bekend staan om hun zure smaak en verfrissende aroma. Ze zijn rijk aan vitamine C en bevatten ook antioxidanten die goed zijn voor de gezondheid. Limoenen worden vaak gebruikt in verschillende gerechten en drankjes, zoals limonade, cocktails en desserts. Ze zijn ook een populaire smaakmaker in de keuken en worden vaak gebruikt om een frisse en zure toets aan gerechten toe te voegen.'

In [23]:
# Making use of the `stream` method
for token in chain.stream({"topic": "lemons", "language": "Dutch"}):
  print(token, end="")

Lemons, ook wel citroenen genoemd, zijn gele vruchten die behoren tot de citrusfamilie. Ze zijn zuur van smaak en staan bekend om hun verfrissende aroma. Lemons bevatten veel vitamine C en worden vaak gebruikt in de keuken als smaakmaker in gerechten, drankjes en desserts. Daarnaast hebben ze ook verschillende gezondheidsvoordelen, zoals het versterken van het immuunsysteem en het bevorderen van de spijsvertering.

This works since, with the introduction of the LangChain Expression Language, all the core abstractions LangChain provides (models, prompts, parsers, retrievers, etc.) all implement the `Runnable` protocol and thus guarantee support for `invoke`, `batch`, `stream`, as well as asynchronous counterparts for these. 

### Explicit Input & Output Schema

What else do Runnables offer that is of value for us?

A: They have an explicit schema definition of what they expect as input and output.

In [24]:
chain.input_schema.schema_json()
# Our chain expects a dictionary with `language` and `topic` attributes as input

'{"title": "PromptInput", "type": "object", "properties": {"language": {"title": "Language", "type": "string"}, "topic": {"title": "Topic", "type": "string"}}}'

In [25]:
chain.output_schema.schema_json()
# And will return a result of type `string`

'{"title": "StrOutputParserOutput", "type": "string"}'

In [26]:
# We can also expect the schema of an interim part of our chain
(prompt | model).output_schema.schema_json()
# The model is guaranteed to return a Result of type AIMessage, HumanMessage, SystemMessage, ChatMessage, FunctionMessage or ToolMessage.
# Usually it will be AIMessage

'{"title": "ChatOpenAIOutput", "anyOf": [{"$ref": "#/definitions/AIMessage"}, {"$ref": "#/definitions/HumanMessage"}, {"$ref": "#/definitions/ChatMessage"}, {"$ref": "#/definitions/SystemMessage"}, {"$ref": "#/definitions/FunctionMessage"}, {"$ref": "#/definitions/ToolMessage"}], "definitions": {"AIMessage": {"title": "AIMessage", "description": "A Message from an AI.", "type": "object", "properties": {"content": {"title": "Content", "anyOf": [{"type": "string"}, {"type": "array", "items": {"anyOf": [{"type": "string"}, {"type": "object"}]}}]}, "additional_kwargs": {"title": "Additional Kwargs", "type": "object"}, "type": {"title": "Type", "default": "ai", "enum": ["ai"], "type": "string"}, "example": {"title": "Example", "default": false, "type": "boolean"}}, "required": ["content"]}, "HumanMessage": {"title": "HumanMessage", "description": "A Message from a human.", "type": "object", "properties": {"content": {"title": "Content", "anyOf": [{"type": "string"}, {"type": "array", "items

## A simple RAG Implementation

RAG stands for Retrieval Augmented Generation and currently is the de-facto architectural pattern to use for grounding an LLM system in custom data. The idea is that we provide the model with the relevant context to answer the question and guide it to use (only) that context for the answer generation.

In order to obtain the relevant fragments of context, a special form of representation (embeddings) is used, that enables a semantic proximity search to obtain the fragments from the corpus that are most relevant for answering the question at hand.

To implement RAG we can re-use many LangChain building blocks we are familiar with already.
> We have already seen the usage of prompts, models and output parsers.
 
In addition, we now introduce the following new building blocks:

Embeddings – these allow us to transform text to a corresponding semantically searchable representation
Chroma – a Vector Database for storing embeddings, as well as easy semantic search & retrieval

We also need a way to "preserve" certain attributes as they flow through the chain.  
For example we now need to utilize the **question** string twice. Once as input to our retriever for identifying the semantically most relevant pieces of context information. And another time, to pass the question to the model, together with the context, for answer generation.

LangChain allows us to do this using either `RunnablePassthrough`, or `operator.itemgetter`. 
I find the usage of `operator.itemgetter` more explicit and easier to reason about and use it in the example below: 

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from operator import itemgetter

# Embed a few sentences as a test
vectorstore = Chroma.from_texts(
    ["Arno loves Python", 
     "Fran loves Wool",
     "Tais enjoys separation of concerns.",
     "Squash makes Corine happy.",
     "Hans likes Philosophy."],
    collection_name="demo-data",
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
# Note: For details on search kwargs see
# https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.as_retriever

# RAG prompt
prompt = ChatPromptTemplate.from_template("""
Answer the question based only on the following context:
{context}

Question: {question}
""")

# LLM
model = ChatOpenAI()

# RAG chain
chain = (
        {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        } 
        | prompt 
        | model 
        | StrOutputParser()
)


In [33]:

# Let's verify that the retriever can obtain the relevant context fragments
retriever.invoke("What does Corine like?")

[Document(page_content='Squash makes Corine happy.'),
 Document(page_content='Squash makes Corine happy.')]

In [34]:
# Optionally, to observe what is happening at each step of the chain, we can set langchain debug mode to `True` 
from langchain.globals import set_debug
# set_debug(True)

In [35]:
chain.invoke({"question": "What does Corine like?"})

'Corine likes squash.'

If we wish to simply pass in a question, without the dictionary we can use the trick from before, using `RunnableParallel` and `RunnablePassthrough` to convert a string into the required shape as part of the chain. 

We also see that the chain objects themselves are composable since they, too, are of type `Runnable`. So if we have more complicated logic, we can break it down into smaller sub-chains that are easier to test and fine-tune in isolation and then use them to construct the final chain:

In [36]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

chain_v2 = (
    RunnableParallel(question=RunnablePassthrough()) # This converts string to dict
    | 
    chain
)

In [37]:
chain_v2.invoke("What does Arno like?")

'Arno likes Python.'

This concludes the material covered in the Dec '23 PAIWeb #1 Meetup.  
The material will be made available on GitHub and also shared in our Zulip community.

If you have questions or would like to explore the topics covered more deeply, 
feel warmly welcome to ask and follow-up questions and engage in conversation on Zulip. :-)

I am looking forward to what we can learn and build together! 